In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import utils
import os
import pandas as pd
import json

In [ ]:
DATASET = 'french_royalty'
rules = ['child','parent','spouse','brother','sister','grandparent', 'full_data']

In [ ]:
counts = dict(zip(rules,np.zeros(len(rules))))
counts['UNK_REL'] = 0

for rule in rules:
    
    data = np.load(os.path.join('..','data',DATASET+'.npz'))

    triples,traces,weights,entities,relations = utils.get_data(data,rule)
    
    num_triples = triples.shape[0]
    
    print(f'{rule}: {num_triples}')
    counts[rule] += triples.shape[0]
    counts[rule + '_num_entities'] = len(entities)

    for i in range(len(traces)):

        for j in range(len(traces[i])):
            
            for k in range(len(traces[i,j])):
                
                rel = traces[i,j,k][1]
                
                counts[rel] += 1
            
#             rels = traces[i,j][:,1]
        
#             if (rels == 'UNK_REL').sum() != 2 :

#                 rels = '_'.join(rels)
                
#                 if rels in counts:
#                     counts[rels] += 1
#                 else:
#                     counts[rels] = 1
#     print('\n')              
#     print(f'rule {rule} counts: {counts}')
#     print('\n')

In [ ]:
counts

In [ ]:
total = 0
for rule in rules:
    total += counts[rule]
total - counts['full_data']

In [ ]:
(traces[i][0] != unk).all(axis=1)

In [ ]:
unk = np.array([['UNK_ENT', 'UNK_REL', 'UNK_ENT']])

In [ ]:
327 / (327 + 41 + 23)

In [ ]:
# num_true_triples = []

# for i in range(len(traces)):
#     num_explanations = 0
#     for j in range(len(traces[i])):

#         current_trace = traces[i][j]

#         num_triples = (current_trace != unk).all(axis=1).sum()

#         if  num_triples > 0:
#             num_explanations += 1
#             num_true_triples.append(num_triples)

In [2]:
DATASET = 'french_royalty'

In [83]:
RULE = 'brother'    

data = np.load(os.path.join('..','data',DATASET+'.npz'))

triples,traces,weights,entities,relations = utils.get_data(data,RULE)

In [85]:
weights[0]

array([['0.3', '0.3'],
       ['0.3', '0.3'],
       ['0.3', '0.3'],
       ['0.9', '0.9'],
       ['0.9', '0.9'],
       ['0.8', 'UNK_WEIGHT'],
       ['UNK_WEIGHT', 'UNK_WEIGHT'],
       ['UNK_WEIGHT', 'UNK_WEIGHT'],
       ['UNK_WEIGHT', 'UNK_WEIGHT'],
       ['UNK_WEIGHT', 'UNK_WEIGHT'],
       ['UNK_WEIGHT', 'UNK_WEIGHT'],
       ['UNK_WEIGHT', 'UNK_WEIGHT'],
       ['UNK_WEIGHT', 'UNK_WEIGHT'],
       ['UNK_WEIGHT', 'UNK_WEIGHT'],
       ['UNK_WEIGHT', 'UNK_WEIGHT'],
       ['UNK_WEIGHT', 'UNK_WEIGHT'],
       ['UNK_WEIGHT', 'UNK_WEIGHT'],
       ['UNK_WEIGHT', 'UNK_WEIGHT']], dtype='<U32')

In [86]:
explaine_data = np.load(
    os.path.join('..','data','preds',DATASET,
        'explaine_'+DATASET+'_'+RULE+'_preds.npz'),allow_pickle=True)

In [87]:
explaine_test_idx = explaine_data['test_idx']

explaine_true_exps = traces[explaine_test_idx]
explaine_true_weights = weights[explaine_test_idx]

explaine_preds = explaine_data['preds']

In [96]:
i = 0
explaine_true_exp = explaine_true_exps[i]
explaine_pred = explaine_preds[i]
true_weight = explaine_true_weights[i]

In [107]:
(np.array(jaccards) == max(jaccards)) * all_weights

array([0. , 0.6, 0. , 0. , 1.8, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. ])

In [110]:
327 / (327 + 17 + 47)

0.8363171355498721

In [99]:
current_traces = explaine_true_exp
pred_exp = explaine_pred
jaccards = []
all_weights = []

for i in range(len(current_traces)):
    
    true_exp = utils.remove_padding_np(current_traces[i],'UNK_ENT','UNK_REL')

    weight = true_weight[i][true_weight[i] != "UNK_WEIGHT"]
    
    sum_weights = sum([float(num) for num in weight])

    all_weights.append(sum_weights)

    jaccard = utils.jaccard_score_np(true_exp, pred_exp)

    jaccards.append(jaccard)

In [7]:
n = len(explaine_pred)

unk = np.array([['UNK_ENT', 'UNK_REL', 'UNK_ENT']])

In [8]:
num_true_triples = []
num_explanations = 0

for i in range(len(explaine_true_exp)):

    current_trace = explaine_true_exp[i]

    num_triples = (current_trace != unk).all(axis=1).sum()

    if  num_triples > 0:
        
        num_explanations += 1
        num_true_triples.append(num_triples)

In [9]:
relevance_scores = np.zeros(num_explanations)

In [10]:
for i in range(n):

    current_pred = explaine_pred[i]

    for j in range(num_explanations):

        unpadded_traces = utils.remove_padding_np(explaine_true_exp[j],'UNK_ENT','UNK_REL')

        unpadded_weights = true_weight[j][true_weight[j] != 'UNK_WEIGHT']

        indices = (unpadded_traces == current_pred).all(axis=1)

        sum_weights = sum([float(num) for num in unpadded_weights[indices]])

        relevance_scores[j] += sum_weights

In [11]:
num_true_triples = np.array(num_true_triples)

In [17]:
prec = relevance_scores / (n * .9)

In [18]:
recal = relevance_scores / (num_true_triples * .9)

In [30]:
np.max(2*(prec * recal) / (prec + recal + .000001))

0.5925921481484815

In [40]:
np.array([0.9,0.9,0,0,0.8]) == 0.9

array([ True,  True, False, False, False])

In [ ]:
triples[:,1] == 'spouse'

In [ ]:
explaine_data = np.load(
    os.path.join('..','data','preds',DATASET,
        'explaine_'+DATASET+'_'+RULE+'_preds.npz'),allow_pickle=True)

In [ ]:
explaine_test_idx = explaine_data['test_idx']
explaine_true_triples = triples[explaine_test_idx]
explaine_true_exps = traces[explaine_test_idx]
explaine_preds = explaine_data['preds']

In [ ]:
i = 102
print(explaine_true_triples[i])
print(explaine_preds[i])

In [ ]:
d = {'spouse_spouse': 327, 'spouse_child_child': 41, 'spouse_child_parent': 23}

In [ ]:
keys = ['_'.join(k.split('_')[1:]) for k,_ in d.items()]
values = list(d.values())


fig, ax = plt.subplots(figsize=(8,4))
ax.bar(keys,values)
ax.set_xticklabels(labels=keys,rotation = (45), fontsize = 14)

In [ ]:
gnn_data = np.load(
    os.path.join('..','data','preds',DATASET,
        'gnn_explainer_'+DATASET+'_'+RULE+'_preds.npz'),allow_pickle=True)

In [ ]:
UNK_ENT_ID = 'UNK_ENT'
UNK_REL_ID = 'UNK_REL'
UNK_WEIGHT_ID = 'UNK_WEIGHT'
MAX_TRACE = data['max_trace']

In [ ]:
gnn_test_idx = gnn_data['test_idx']
gnn_true_triples = triples[gnn_test_idx]
gnn_true_exps = traces[gnn_test_idx]
gnn_true_weights = weights[gnn_test_idx]

gnn_preds = gnn_data['preds']

num_gnn_triples = gnn_true_exps.shape[0]

In [ ]:
predicate_counts = {}
correct_counts = {}
for i in range(num_gnn_triples):
    
    gnn_true_triple = gnn_true_triples[i]
    gnn_true_exp = gnn_true_exps[i]
    gnn_pred = gnn_preds[i]
    true_weight = gnn_true_weights[i]
    
    max_jaccard,max_idx = utils.max_jaccard_np(gnn_true_exp,gnn_pred,UNK_ENT_ID,UNK_REL_ID,return_idx=True)
    
    max_predicates = gnn_true_triple[1] +'_' + '_'.join([p for p in gnn_true_exp[max_idx,:,1] if p != 'UNK_REL'])
    
    if max_jaccard < 1:

        if max_predicates in predicate_counts:
            predicate_counts[max_predicates] += 1
        else:
            predicate_counts[max_predicates] = 1
            
    else:
        
        if max_predicates in correct_counts:
            correct_counts[max_predicates] += 1
        else:
            correct_counts[max_predicates] = 1

In [ ]:
with open(os.path.join('..','data','predicate_weights.json'),'r') as f:
    predicate_weights = json.load(f)    

In [ ]:
weight_counts = {}

for k,count in predicate_counts.items():
    
    str_weight = predicate_weights[k]
    
    if str_weight in weight_counts:
        weight_counts[str_weight] += count
    else:
        weight_counts[str_weight] = count

In [ ]:
correct_weight_counts = {}

for k,count in correct_counts.items():
    
    str_weight = predicate_weights[k]
    
    if str_weight in correct_weight_counts:
        correct_weight_counts[str_weight] += count
    else:
        correct_weight_counts[str_weight] = count

In [ ]:
weight_counts

In [ ]:
# weight_sum = []

# for i in range(len(weights[0])):
    
#     summm = 0.0
    
#     for j in weights[0][i]:
        
#         if j != 'UNK_WEIGHT':
#             summm += float(j)
#     weight_sum.append(summm)

# weight_sum

In [ ]:
set(predicate_weights.values())

In [ ]:
#['UNK_REL', 'child', 'parent', 'spouse']

In [ ]:


# explaine_data = np.load(
#     os.path.join('..','data','preds',DATASET,
#         'explaine_'+DATASET+'_'+RULE+'_preds.npz'),allow_pickle=True)

In [ ]:
#triples[(triples[:,0] == '<http://dbpedia.org/resource/Louis_VII_of_France>')][12]

In [ ]:
# [['<http://dbpedia.org/resource/Agnes_of_France,_Byzantine_Empress>',
#          'grandparent',
#          '<http://dbpedia.org/resource/Louis_VI_of_France>'],
#         ['<http://dbpedia.org/resource/Louis_VII_of_France>', 'parent',
#          '<http://dbpedia.org/resource/Louis_VI_of_France>']],

# array([[['<http://dbpedia.org/resource/Adela_of_Champagne>', 'child',
#          '<http://dbpedia.org/resource/Agnes_of_France,_Byzantine_Empress>'],
#         ['<http://dbpedia.org/resource/Louis_VII_of_France>', 'spouse',
#          '<http://dbpedia.org/resource/Adela_of_Champagne>']],


In [ ]:
#traces[(triples[:,0] == '<http://dbpedia.org/resource/Louis_VII_of_France>')][12]

In [ ]:
import RGCN

RULE = 'full_data'

full_data = np.load(os.path.join('..','data',DATASET+'.npz'))

full_triples,full_traces,full_weights,full_entities,full_relations = utils.get_data(full_data,RULE)

full_model = RGCN.get_RGCN_Model(
    num_entities=len(full_entities),
    num_relations=len(full_relations),
    embedding_dim=10,
    output_dim=10,
    seed=123
)

NUM_ENTITIES_FULL = len(full_entities)
NUM_RELATIONS_FULL = len(full_relations)

full_ent2idx = dict(zip(full_entities, range(NUM_ENTITIES_FULL)))
full_rel2idx = dict(zip(full_relations, range(NUM_RELATIONS_FULL)))

full_model.load_weights(os.path.join('..','data','weights',DATASET,DATASET+'_'+RULE+'.h5'))

In [ ]:
full_embeddings = full_model.get_layer('entity_embeddings').get_weights()[0]

In [ ]:
RULE = 'full_data'

data = np.load(os.path.join('..','data',DATASET+'.npz'))

triples,traces,weights,entities,relations = utils.get_data(data,RULE)

model = RGCN.get_RGCN_Model(
    num_entities=len(entities),
    num_relations=len(relations),
    embedding_dim=10,
    output_dim=10,
    seed=123
)


NUM_ENTITIES = len(entities)
NUM_RELATIONS = len(relations)

ent2idx = dict(zip(entities, range(NUM_ENTITIES)))
rel2idx = dict(zip(relations, range(NUM_RELATIONS)))

model.load_weights(os.path.join('..','data','weights',DATASET,DATASET+'_'+RULE+'.h5'))

In [ ]:
spouse_embeddings = model.get_layer('entity_embeddings').get_weights()[0]

In [ ]:
intersect, comm1, comm2 = np.intersect1d(full_entities,entities,return_indices=True)

In [ ]:
XX = np.sum(np.square(full_embeddings[comm1]), axis=1)[:,np.newaxis]
YY = np.sum(np.square(spouse_embeddings), axis=1)

distances = XX + YY - 2 * np.dot(full_embeddings[comm1], spouse_embeddings.T)

In [ ]:
np.mean(distances)

In [ ]:
from SPARQLWrapper import SPARQLWrapper, XML
from rdflib import Graph

In [ ]:
limit = 1000
i = 0

In [ ]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery(f"""
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    CONSTRUCT{{
    ?x
    dbo:child ?child .
    }}
    WHERE {{
    ?x a dbo:Royalty .
    OPTIONAL {{?x dbo:child ?child }}
    }}
    """)


In [ ]:
sparql.setReturnFormat(XML)

In [ ]:
results = sparql.query().convert().serialize(destination='/Users/nhalliwe/Desktop/test',
            format='xml'
            )

In [ ]:
# PREFIX dbo: <http://dbpedia.org/ontology/>
# PREFIX foaf: <http://xmlns.com/foaf/0.1/>

# CONSTRUCT{ 
#      ?x dbo:parent ?parent ;
#      dbo:child ?child ;
#      dbo:spouse ?spouse ;
#      foaf:gender ?gender ;
#  }
# WHERE {
#   OPTIONAL {?x dbo:parent ?parent }
#   OPTIONAL {?x dbo:spouse ?spousea }
#   OPTIONAL {?x foaf:gender ?gender }
#   OPTIONAL {?x dbo:child ?child } 

#   FILTER regex(?x,"England")
# }
#rename file with .ttl

In [ ]:
# PREFIX dbo: <http://dbpedia.org/ontology/>
# PREFIX foaf: <http://xmlns.com/foaf/0.1/>

# CONSTRUCT{ 
#      ?x dbo:parent ?parent ;
#      dbo:child ?child ;
#      dbo:spouse ?spouse ;
#      foaf:gender ?gender ;
#  }
# WHERE {
#   ?x dbo:parent ?parent .
#   ?x dbo:spouse ?spouse .
#   ?x foaf:gender ?gender .
#   ?x dbo:child ?child .

#   FILTER regex(?x,"England")
# }
#small english

In [ ]:
from rdflib import Graph
from rdflib import plugin
import rdflib

In [ ]:
plugin.register(
    'sparql', rdflib.query.Processor,
    'rdfextras.sparql.processor', 'Processor')
plugin.register(
    'sparql', rdflib.query.Result,
    'rdfextras.sparql.query', 'SPARQLQueryResult')

In [ ]:
g = Graph(base='http://www.w3.org/2000/01/rdf-schema#')

In [ ]:
g.parse('/Users/nhalliwe/Desktop/full_test',format='xml')

In [ ]:
query = """
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    
CONSTRUCT{{
    ?x
    dbo:parent ?parent ;
    dbo:child ?child ;
    dbo:spouse ?spouse ;
    foaf:gender ?gender .
    }}
WHERE {{
    ?x a dbo:Royalty .
    OPTIONAL {{?x dbo:parent ?parent }}
    OPTIONAL {{?x dbo:spouse ?spouse }}
    OPTIONAL {{?x foaf:gender ?gender }}
    OPTIONAL {{?x dbo:child ?child }}
"""

In [ ]:
# for row in g.query(query):µ
#     print(row['*'])

g.query(query)#.serialize(destination='/Users/nhalliwe/Desktop/test',format='xml')

In [ ]:
#g.parse('/Users/nhalliwe/Desktop/Explain-KG/data/rules/full_royalty',format='xml')

In [ ]:
#g.parse('/Users/nhalliwe/Desktop/test',format='xml')

In [ ]:
#g.serialize(destination='/Users/nhalliwe/Desktop/full_test',format='xml')

In [ ]:
# triple_lookup = {}
# longest_trace = -1
# max_padding = 3

# for i in unique_idx:
    
#     triple = triples[i]
    
#     indices = (triples == triple).all(axis=1)
        
#     triple_lookup[str(triple)] = indices
    
#     sum_indices = indices.sum()
    
#     if sum_indices > longest_trace:
        
#         longest_trace = sum_indices

In [ ]:
# processed_triples = []
# processed_weights = []
# processed_traces = []
# unk = np.array([['UNK_ENT','UNK_REL','UNK_ENT']])
# unk_weight_str = 'UNK_WEIGHT'
# unk_weight = np.array([[unk_weight_str] * max_padding])

# for idx in unique_idx:
    
#     triple = triples[idx]
    
#     trace_indices = triple_lookup[str(triple)]
#     trace = traces[trace_indices]
#     weight = weights[trace_indices]
    
#     per_trace_weights = []

#     for i in range(len(trace)):

#         num_triples = trace[i].shape[0]
#         current_weight = weights[trace_indices][i]

#         num_unk = (trace[i] == unk).all(axis=1).sum()

#         current_weights = [current_weight] * (num_triples-num_unk)

#         while len(current_weights) != num_triples:

#             current_weights.append(unk_weight_str)
            
#         per_trace_weights.append(current_weights)
          
#     per_trace_weights = np.array(per_trace_weights)
    
#     while per_trace_weights.shape[0] != longest_trace:
#         per_trace_weights = np.concatenate([per_trace_weights,unk_weight],axis=0)
        
#     padded_trace = utils.pad_trace(trace,max_padding=max_padding,longest_trace=longest_trace,unk=unk)
    
#     processed_triples.append(triple)
#     processed_traces.append(padded_trace)
#     processed_weights.append(per_trace_weights)

In [ ]:
# all_triples = np.array(processed_triples)
# all_traces = np.array(processed_traces)
# all_weights = np.array(processed_weights)
#traces: (NUM_TRIPLES,LONGEST_TRACE,MAX_PADDING,3)
#weights: (NUM_TRIPLES,LONGEST_TRACE,MAX_PADDING)

In [ ]:
# idx = 2
# current_traces = all_traces[idx]
# current_weights = all_weights[idx]

### pred_exp = np.array([['<http://example.org/data#MotherPaul>', 'child',
#          '<http://example.org/data#BrotherPaul>'],
#         ['<http://example.org/data#FatherPaul>', 'child',
#          '<http://example.org/data#BrotherPaul>']
#         ])

# pred_exp = np.array(
#     [['<http://example.org/data#MotherPaul>', 'spouse',
#          '<http://example.org/data#FatherPaul>']])
# def precision_recall(pred_exp,current_traces,current_weights):
    
#     n = len(pred_exp)

#     relevance_scores = np.zeros(longest_trace) #numerator of graded recall

#     for i in range(n):

#         current_pred = pred_exp[i]

#         for j in range(len(current_traces)):

#             unpadded_traces = remove_padding_np(current_traces[j],'UNK_ENT','UNK_REL')
#             unpadded_weights = current_weights[j][current_weights[j] != 'UNK_WEIGHT']

#             indices = (unpadded_traces == current_pred).all(axis=1)

#             sum_weights = sum([float(num) for num in unpadded_weights[indices]])

#             relevance_scores[j] += sum_weights

#     max_relevance_score = max(relevance_scores)
#     max_idx = np.argmax(relevance_scores)

#     total_sum = sum([float(weight) for weight in current_weights[max_idx] if weight != 'UNK_WEIGHT'])

#     precision = max_relevance_score/n
#     recall = max_relevance_score/total_sum
    
#     return precision, recall

In [ ]:
#d1,d2,_ = (all_traces[2] != ['UNK_ENT','UNK_REL','UNK_ENT']).nonzero()
#all_traces[2][d1,d2]

In [ ]:
#import tensorflow as tf

In [ ]:
#tf_pred_exp = tf.convert_to_tensor(pred_exp)

In [ ]:
#tf_traces_i = tf.convert_to_tensor(all_traces[2])

In [ ]:
#triples2idx = utils.array2idx(all_triples,ent2idx,rel2idx)

In [ ]:
#traces2idx = utils.array2idx(all_traces,ent2idx,rel2idx)

In [ ]:
#import RGCN

In [ ]:
# model = RGCN.get_RGCN_Model(
#     num_entities=NUM_ENTITIES,
#     num_relations=NUM_RELATIONS,
#     embedding_dim=10,
#     output_dim=10,
#     seed=123
# )

# model.load_weights(os.path.join('..','data','weights',DATASET,DATASET+'_'+RULE+'.h5'))

In [ ]:
# ALL_INDICES = tf.reshape(tf.range(0,NUM_ENTITIES,1,dtype=tf.int64), (1,-1))

# ADJACENCY_DATA = tf.concat([triples2idx,traces2idx.reshape(-1,3)],axis=0)
# adj_mats = utils.get_adj_mats(ADJACENCY_DATA,NUM_ENTITIES,NUM_RELATIONS)